Randomモジュールをインポート

In [212]:
import random

### ネイピア数
$$e = \lim_{{x \to \infty}} \left(1 + \frac{1}{x}\right)^x$$

In [213]:
def napiers_logarithm(x):
    return (1 + 1 / x) ** x
napier_number = napiers_logarithm(100000000)  # e

### シグモイド関数
$$Sigmoid(x)=\frac{1}{1+Napiers(1000000000)^{-x}}$$

In [214]:
def sigmoid(x):
    return 1 / (1 + napier_number ** -x)

### シグモイド関数の微分
$$Sigmoid'(x)=Sigmoid(x)(1 - Sigmoid(x))$$

In [215]:
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

### ReLU関数
$$ReLU(x) =
        \begin{cases}
            x \quad x \geqq 0 \\
            0 \quad x < 0 \\
        \end{cases}
$$

In [216]:
def relu(x):
    return max(0, x)

### ReLU関数の微分
$$ReLU'(x) =
        \begin{cases}
            1 \quad x > 0 \\
            0 \quad x \leqq 0 \\
        \end{cases}
$$

In [217]:
def relu_derivative(x):
    return 1 if x > 0 else 0

### 自然対数
$$ln(x) = 2 \sum_{n=1,3,5,\dots}^{\infty} \frac{(z^n)}{n}, \quad z = \frac{x-1}{x+1}$$

In [218]:
def ln(x, n_terms=100):
    if x <= 0: raise ValueError("x must be positive")
    z = (x - 1) / (x + 1)
    result = 0
    for n in range(1, n_terms + 1, 2):
        result += (z ** n) / n
    return 2 * result

### クロスエントロピー損失
$$L(y_{\text{true}}, y_{\text{pred}}) = - \sum_{i=1}^{n} y_{\text{true}_i} \cdot \ln(y_{\text{pred}_i} + \epsilon)$$

In [219]:
def cross_entropy_loss(y_true, y_pred):
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    return -sum([t * ln(p + 1e-9) for t, p in zip(y_true, y_pred)])

### ニューラルネットワークを初期化
例:&emsp;入力層 -> 2,&emsp;隠れ層 -> [3, 3],&emsp;出力層 -> 1
#### 重み
$$n_{n}={2, 3, 3, 1}$$
$$ random \in [-1, 1] $$
$$[[[random]*n_{1}]*n_{2}, [[random]*n_{2}]*n_{3}, [[random]*n_{3}]*n_{4}]$$
#### バイアス
$$n_{n}={3, 3, 1}$$
$$ random \in [-1, 1] $$
$$[[random]*n_{1},[random]*n_{2},[random]*n_{3}]$$

※ [n]*5 -> [n, n, n, n, n]

In [220]:
def initialize_weights(layer_sizes):  # initialize weights and biases
    weights, biases = [], []
    for i in range(len(layer_sizes) - 1):
        W = [[random.uniform(-1, 1) for _ in range(layer_sizes[i])] for _ in range(layer_sizes[i+1])]  # [ {random_num *layer_sizes[i]} *layer_sizes[i+1] ]
        b = [random.uniform(-1, 1) for _ in range(layer_sizes[i+1])]  # [ random_num *layer_sizes[i+1] ]
        weights.append(W)
        biases.append(b)
    return weights, biases

In [221]:
def forward_propagation(inputs, weights, biases):  # forward propagation
    activations = [inputs]
    for W, b in zip(weights, biases):
        z = [
            sum([activations[-1][i] * W[j][i] for i in range(len(activations[-1]))]) + b[j]
            for j in range(len(b))
        ]
        activations.append([relu(z_i) for z_i in z])
    return activations

In [222]:
def backward_propagation(activations, y_true, weights, biases, learning_rate):  # backward propagation
    output_layer = activations[-1]
    errors = [
        (output_layer[i] - y_true[i]) * sigmoid_derivative(output_layer[i])
        for i in range(len(y_true))
    ]
    deltas = [errors]
    # Backpropagating the hidden layer error
    for l in range(len(weights)-1, 0, -1):
        hidden_errors = [
            sum([deltas[0][k] * weights[l][k][j] for k in range(len(deltas[0]))]) * relu_derivative(activations[l][j])
            for j in range(len(activations[l]))
        ]
        deltas.insert(0, hidden_errors)
    # Update the weights and biases
    for l in range(len(weights)):
        for i in range(len(weights[l])):
            for j in range(len(weights[l][i])):
                weights[l][i][j] -= learning_rate * deltas[l][i] * activations[l][j]
            biases[l][i] -= learning_rate * deltas[l][i]

    return weights, biases

In [223]:
def train(X, y, layer_sizes, epochs, learning_rate):
    weights, biases = initialize_weights(layer_sizes)

    for epoch in range(epochs):
        total_loss = 0
        for i in range(len(X)):
            activations = forward_propagation(X[i], weights, biases)
            total_loss += cross_entropy_loss(y[i], activations[-1])
            weights, biases = backward_propagation(activations, y[i], weights, biases, learning_rate)
        
        m = (epoch + (epochs // 20) - 1) // (epochs // 20)
        print(f"\rEpoch {epoch+1}/{epochs}, Loss: {total_loss/len(X)}, [{'+'*m}{' '*(20-m)}]{' '*10}",end="")
    print("\nComplete")
    return weights, biases

In [231]:
# DataSet for XOR
X = [[0, 0], [0, 1], [1, 0], [1, 1]]  # Input
y = [[0], [1], [1], [0]]  # Output

epochs = 500  # Number of epochs
learning_rate = 0.01  # Learning rate
layer_sizes = [2, 8, 16, 8, 1]  # 2 input -> 8 hidden -> 16 hidden -> 8 hidden -> 1 output

weights, biases = train(X, y, layer_sizes, epochs, learning_rate)

for i in range(len(X)):  # Prediction
    activations = forward_propagation(X[i], weights, biases)
    output = activations[-1]
    binary_output = [1 if o >= 0.5 else 0 for o in output]
    print(f"Inputs: {X[i]}, Output: {y[i]} Predict: {binary_output}, probability: {round(output[0], 3)}")

Epoch 500/500, Loss: 0.005309353010232144, [++++++++++++++++++++]           
Complete
Inputs: [0, 0], Output: [0] Predict: [0], probability: 0.02
Inputs: [0, 1], Output: [1] Predict: [1], probability: 0.981
Inputs: [1, 0], Output: [1] Predict: [1], probability: 0.999
Inputs: [1, 1], Output: [0] Predict: [0], probability: 0.006
